# Gamma Matrix Estimation

In this notebook, I am going to use the pyGSTi data to estimate the response matrix for measurement process. 

## I. Introduction of Background Knowledge
The $\Gamma$ matrix here in our discussion is a kind of response matrix without the state-preparation error. In general calibration experiment, we need to prepare every possible basis state for $n$ qubits experiment, and measure the output result with different input basis state. Based on the measurement result, we can define the response matrix $A$, 
\begin{equation}
p_{\rm exp} = Ap_{\rm ideal}
\end{equation}
where $p_{\rm exp}$ and $p_{\rm ideal}$ are the probability of each basis state from experiment and theory, respectively. The element of response matrix $A$ is given by
\begin{equation}
a_{ij} = Pr(\text{measure }i|\text{true }j)
\end{equation}

However, the initial state is not precisely prepared as expected, so from the final measurement we will obtain a response matrix with state-preparation error. Namely, the element of $A$ matrix is not precisely $Pr(\text{measure }i|\text{true }j)$. To consider the state-preparation error, Michael Geller purposes a method using gate-set tomography (GST) to estimate $Pr(\text{measure }i|\text{true }j)$. (see here: [Conditionally rigorous mitigation of multiqubit measurement errors](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.127.090502)). We call the response matrix estimated from GST techniques $\Gamma$ matrix to distinguish the response matrix $A$ obtained from traditional calibration experiment. The estimation for one qubit will mainly follow the procedure below:

- Perform calibration experiment
    - Prepare $|0\rangle$, $|1\rangle$, $|+\rangle$ and $|\psi\rangle = SH|0\rangle$ for one qubit
    - Get the measurement result for every qubit
- Perform GST experiment
- Estimate the $\Gamma$ matrix
    - Estimate noisy state $\rho_0$, $\rho_1$, $\rho_{+}$ and $\rho_{+i}$ via GST experiment result
    - Estimatie $L$ matrix from noisy state and noisy $G_{x}$ and $G_{y}$ gate
    - Estimate the $\Gamma$ matrix via $L$ matrix
- Perform error mitigation with $\Gamma$ matrix

In the following sections, I will provide details of the third step. Please refer to other notebook for details about the first, second and last step. 

In [2]:
# Import require package
import numpy as np
from qiskit import IBMQ
# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

## II. The Procedure of $\Gamma$ Matrix Estimation
In this section, I will describe details of how to finish 4 steps of $\Gamma$ matrix estimation. 

### A. Perform Calibration Experiment
The calibration experiment is used to collect required data from the device to estimate the bias of measurement. Please refer to other notebook for details. 

### B. Perform GST Experiment
The GST experiment is used to collect required data from the device to estimate the noisy prepared state. Please refer to other notebook for details. 

### C. Estimate $\Gamma$ Matrix
In this section, I will use the calibration and GST result to estimate the $\Gamma$ matrix. For illustration, I will focus only on qubit 0 of quantum device ibmq\_lima as an example. We need to prepare four single-qubit state, 

\begin{equation}
|0\rangle = \begin{pmatrix} 1 \\ 0 \end{pmatrix}, 
|1\rangle = \begin{pmatrix} 0 \\ 1 \end{pmatrix}, 
|+\rangle = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 \\ 1 \end{pmatrix},
|\psi\rangle = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 \\ i \end{pmatrix}
\end{equation}
 
which correspond to the density matrix
\begin{equation}
\pi_0 = \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix}, 
|1\rangle = \begin{pmatrix} 0 & 0 \\ 0 & 1 \end{pmatrix}, 
|+\rangle = \frac{1}{2}\begin{pmatrix} 1 & 1\\ 1 & 1 \end{pmatrix},
|\psi\rangle = \frac{1}{2}\begin{pmatrix} 1 & -i\\ i & 1 \end{pmatrix}
\end{equation}

The output of prepare these four state for qubit 0 is given below. 

In [ ]:
try_zero_result = {'get-zero': 505, 'get-one': 495}
try_one_result = {'get-zero': 505, 'get-one': 495}
try_plus_result = {'get-zero': 496, 'get-one': 504}
try_psi_result = {'get-zero': 505, 'get-one': 495}

Due to the presence of state-preparation error, instead of preparing $\{\pi_0, \pi_1, \pi_+, \pi_{+i}\}$, we will prepare a set of noisy state $\{\rho_0, \rho_1, \rho_+, \rho_{+i}\}$. The noisy $\rho_0$ can be directly obtained from GST experiment result, while the other three noisy state can only be estimated by noisy gate $G_{x}$ and $G_y$, with their noise-free form as

\begin{equation}
G_{x} = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & -1 \\
0 & 0 & 1 & 0 \\
\end{pmatrix}, G_{y} = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 \\
0 & -1 & 0 & 0 \\
\end{pmatrix}
\end{equation}

The noisy $\rho_0$, $G_{x}$ and $G_y$ result from GST experiment is shown below.

In [2]:
# Noisy state rho0
rho0 = np.array([[0.9945645, -0.0085467],
                 [-0.0269976, 0.0054355]])

# Noisy gate Gx
Gx = np.array([[1, 0, 0, 0], 
               [0.0003134, 0.9975053, 0.0070664, 0.0061008], 
               [-0.003582, 0.0017868, -0.0002412, -0.9982639], 
               [0.0199663, -0.0008216, 0.9994638, 0.0049076]])

# Noisy gate Gy
Gy = np.array([[1, 0, 0, 0], 
               [-0.0035356, -0.0031584, 0.0092401, 0.9983456], 
               [0.0001654, 0.0035338, 0.9994461, 0.0013525], 
               [0.0113752, -0.9985736, 0.0043543, 0.0006209]])

Gx2 = np.matmul(Gx, Gx)
Gx3 = np.matmul(Gx2, Gx)

With data of noisy state and noisy gate, I will estimate the matrix $L$ below. The matrix $L$ is defined 

In [ ]:
def getLMatrix(state):
    """
    Calculate the L Matrix from noisy density operator
    """
    a = state[0][0]
    b = state[0][1]
    c = state[1][0]
    d = state[1][1]
    
    x = b+c
    y = (b-c)*1j
    z = 2*a - 1
    
    return 0.5*np.array([[1+z, x-y*1j], 
                         [x+y*1j, 1-z]])

### Get GST Vector

In the GST language, one should use the GST vector of a noisy state to perform calculation. In this case, we have
\begin{equation}
|\rho_{\lambda}\rangle\rangle = \frac{1}{2}\begin{pmatrix}
a+d \\
b+c \\
(b-c)i \\
a-d \\
\end{pmatrix}
\end{equation}

In [9]:
def getGSTVector(state):
    a = state[0][0]
    b = state[0][1]
    c = state[1][0]
    d = state[1][1]
    return 0.5*np.array([a+d, b+c, (b-c)*1j, a-d])

In [10]:
def getDenOper(GSTVector):
    a = GSTVector[0]
    b = GSTVector[1]
    c = GSTVector[2]
    d = GSTVector[3]
    return np.array([[a+d, b-c*1j], 
                     [b+c*1j, a-d]])

In [11]:
getDenOper(getGSTVector(rho0))

array([[ 0.9945645+0.j, -0.0085467+0.j],
       [-0.0269976+0.j,  0.0054355+0.j]])

In [13]:
rho0_GST = getGSTVector(rho0)
rho1_GST = np.matmul(Gx2, rho0_GST)
rhop_GST = np.matmul(Gy, rho0_GST)
rhoi_GST = np.matmul(Gx3, rho0_GST)

rho1 = getDenOper(rho1_GST)
rhop = getDenOper(rhop_GST)
rhoi = getDenOper(rhoi_GST)

### Gamma Matrix Estimation

In [11]:
np.zeros(4)

array([0., 0., 0., 0.])

In [ ]:
gamma1 = np.array([[0.9981, 0.0063], [0.0019, 0.9937]])
gamma2 = np.array([[0.9981, 0.0063], [0.0019, 0.9937]])
gamma3 = np.array([[0.9977, 0.0108], [0.0023, 0.9892]])
gamma4 = np.array([[0.9986, 0.0125], [0.0014, 0.9875]])

In [ ]:
tmp1 = np.kron(gamma1, gamma2)
tmp2 = np.kron(gamma3, gamma4)
Gamma = np.kron(tmp1, tmp2)

In [ ]:
Gamma